In [1]:
import xlrd
import xlwt

In [2]:
class Student:
    def __init__(self, name, sex, sciper, email):
        self.name = name
        self.sex = sex
        self.sciper = int(sciper)
        self.email = email
    
    def show(self):
        print(self.sex + "\t" + str(self.sciper) + "\t" + self.name)

In [3]:
def import_file(filename,name=1,sex=0,sciper=10,email=11):
    
    workbook = xlrd.open_workbook(filename)
    sheet = workbook.sheet_by_index(0)
    
    students = []
    for i in range(3,sheet.nrows):
        row = sheet.row(i)
        
        # change empty SCIPER fields to 0
        if (row[sciper].value == 'No Sciper'):
            row[sciper].value = 0
        
        # infer sex from saluation
        if (row[sex].value == "Madame"):
            row[sex].value = "F"
        elif (row[sex].value == "Monsieur"):
            row[sex].value = "M"
        else: 
            row[sex].value = "Sex"
        
        students.append(Student(row[name].value,row[sex].value,row[sciper].value,row[email].value))
    return students

In [4]:
def import_check_headers(filename):
    
    print("\n Importing " + str(filename))
    
    workbook = xlrd.open_workbook(filename)
    
    print("\nThis workbook contains the following sheets:")
    print(workbook.sheet_names())
    sheet_index = int(input("Which sheet contains the data? : "))
    
    sheet = workbook.sheet_by_index(sheet_index)
    
    print("\nThese are the first 5 rows of the sheet : \n")
    for i in range(0,5):
        line = "Line " + str(i) + " : "
        for j in range(0,sheet.ncols):
            line += str(sheet.cell(i,j).value) + "; "
        print(line)
    row = int(input("Which row contains the column headers? : "))
    
    print("\nThese are the headers:")
    for i in range(0,sheet.ncols):
        print(str(i) + " : " + str(sheet.cell(row,i).value))
    print("Which column contains the :")
    name = int(input("Name : "))
    sciper = int(input("SCIPER : "))
    sex = int(input("Sex : "))
    email = int(input("Email : "))

    return import_file(filename,name,sex,sciper,email)

In [5]:
def write_to_file(student_list,filename):
    workbook = xlwt.Workbook()
    sheet = workbook.add_sheet("Sheet1")
    
    sheet.write(0,0,"SCIPER")
    sheet.write(0,1,"Sex")
    sheet.write(0,2,"Name")
    sheet.write(0,3,"email")
    
    i = int(1)
    for student in student_list:
        sheet.write(i,0,student.sciper)
        sheet.write(i,1,student.sex)
        sheet.write(i,2,student.name)
        sheet.write(i,3,student.email)
        i = i + 1
    
    workbook.save(filename)

In [6]:
old_student_list = import_check_headers('s_old.xlsx')
new_student_list = import_check_headers('s_new.xlsx')


 Importing s_old.xlsx

This workbook contains the following sheets:
['Sheet1']
Which sheet contains the data? : 0

These are the first 5 rows of the sheet : 

Line 0 : Ces listes d'étudiants ne seront définitives qu'à la mi-août une fois la commission d'admission bachelor et la conférence d'examen passées afin de s'assurer que tous les étudiants nouvellement admis à l'EPFL y figurent et que les étudiants en situation d'échec définitif soient rayés de ces listes; ; ; ; ; ; ; ; ; ; ; ; 
Line 1 : Sciences et technologies du vivant, 2015-2016, Bachelor semestre 1  (269 ét.); ; ; ; ; ; ; ; ; ; ; ; 
Line 2 : Civilité; Nom Prénom; Orientation Bachelor; Orientation Master; Spécialisation; Filière opt.; Mineur; Statut; Type Echange; Ecole Echange; No Sciper; ; 
Line 3 : Madame; Abdennadher Amna; ; ; ; ; ; Présent; ; ; 247331.0; mailto:; 
Line 4 : Monsieur; Aebi Benjamin Giovanni Numa; ; ; ; ; ; Présent; ; ; 262217.0; mailto:; 
Which row contains the column headers? : 2

These are the headers:


In [15]:
"""
This function will merge an old and new list of students, while
keeping info from the old data.
The merge is based in SCIPER numbering
"""
def merge_students(old_list,new_list):
    merged_list = []
    for new_student in new_list:
        existed = False
        for old_student in old_list:
            if new_student.sciper == old_student.sciper:
                existed = True
                merged_list.append(old_student)
                break
        if existed == False:
            merged_list.append(new_student)
    return merged_list

def find_new_students(old_list,new_list):
    new_students = []
    for new_student in new_list:
        existed = False
        for old_student in old_list:
            if new_student.sciper == old_student.sciper:
                existed = True
                break
        if existed == False:
            new_students.append(new_student)
    
    return new_students

In [16]:
new_student_list = find_new_students(old_student_list, new_student_list)

In [17]:
write_to_file(new_student_list,"new_students.xlsx")